# Q2 (a): Lagrangian dual with Subgradient Optimization

In [286]:
import numpy as np
from numpy import random
from gurobipy import *
step = -4

In [287]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

lamb0,lamb1,lamb2=0,0,0
lamb = np.array([lamb0,lamb1,lamb2])
varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers


In [288]:
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))
# ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
#                  + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j]*demand[ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()

In [289]:
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()



Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1


In [290]:
# Optimize Model (you should see output when running this cell)
ufl.optimize()

Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00


In [291]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 0.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 0.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [292]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 0.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [293]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

In [294]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(2.3094010767585034, 2.3094010767585034, 2.3094010767585034)

In [295]:
# 2

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))
# ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
#                  + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j]*demand[ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()
# Optimize Model (you should see output when running this cell)
ufl.optimize()


Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations 

In [296]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 0.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 0.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [297]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 0.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [298]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

In [299]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(4.618802153517007, 4.618802153517007, 4.618802153517007)

In [300]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()
# Optimize Model (you should see output when running this cell)
ufl.optimize()


Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations 

In [301]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 0.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 0.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [302]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 0.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [303]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

In [304]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(6.9282032302755105, 6.9282032302755105, 6.9282032302755105)

In [305]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))
# ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
#                  + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j]*demand[ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()
# Optimize Model (you should see output when running this cell)
ufl.optimize()


Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations 

In [306]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 0.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 0.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [307]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 0.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [308]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

In [309]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(9.237604307034013, 9.237604307034013, 9.237604307034013)

In [310]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))
# ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
#                  + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j]*demand[ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()
# Optimize Model (you should see output when running this cell)
ufl.optimize()


Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.4256258e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations 

In [311]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 1.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 1.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 1.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 1.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 1.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 1.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [312]:
Open

{0: <gurobi.Var Open0 (value 1.0)>,
 1: <gurobi.Var Open1 (value 1.0)>,
 2: <gurobi.Var Open2 (value 1.0)>}

In [313]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(-0.5773502691896258, -0.5773502691896258, -0.5773502691896258)

In [314]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(6.9282032302755105, 6.9282032302755105, 6.9282032302755105)

In [315]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

varcost = np.array([[10,1,1],[1,10,1],[1,1,10]]) - lamb*np.array([[1,1,1],[1,1,1],[1,1,1]])
demand = np.ones(M) # demand of customers
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # Y, variables corresponding to fraction of demand met
Open = {} # X, indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))
# ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
#                  + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j]*demand[ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

# for ind_j in range(M):
#     MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
#     IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
#                                   M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
    for ind_j in range(M):
        IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()
# Optimize Model (you should see output when running this cell)
ufl.optimize()


Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 9 rows, 12 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations 

In [316]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 0.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 0.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 0.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [317]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 0.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [318]:
d0 = 1-FracMet[(0,0)].X-FracMet[(1,0)].X-FracMet[(2,0)].X
d1 = 1-FracMet[(0,1)].X-FracMet[(1,1)].X-FracMet[(2,1)].X
d2 = 1-FracMet[(0,2)].X-FracMet[(1,2)].X-FracMet[(2,2)].X
norm_k = np.sqrt(d0*d0+d1*d1+d2*d2)
d0,d1,d2=d0/norm_k,d1/norm_k,d2/norm_k
d0,d1,d2

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

In [319]:
lamb0,lamb1,lamb2 = lamb0-step*d0,lamb1-step*d1,lamb2-step*d2
lamb = np.array([lamb0,lamb1,lamb2])
lamb0,lamb1,lamb2

(9.237604307034013, 9.237604307034013, 9.237604307034013)